# Computation of the pullback of the complex gradient of a function to its resolution

Set variables that we are going to use (it's important to tell sage that these are complex variables so that the method ``simplify_full()´´ works well

In [1]:
var('u v x y', domain='complex')

(u, v, x, y)

We have $3$ sections. First we define the simplest function which does not invoke the Hessian. Then we define the "complicated" method using the Hessian. In the final section we have a step-by-step record of the computations to keep track of mistakes.

## 1. Easy function (without Hessian)

It's pretty much selfdescriptive

In [2]:
def pullback(f, p):
    g=f(p[0],p[1]);
    J=jacobian(p,(u,v));
    W = (conjugate(f)^(-1))*(matrix(SR, 2, [conjugate(diff(f,x)),conjugate(diff(f,y))]));
    L=J(u,v)^(-1)*W(p[0],p[1]);
    return L.simplify_full()

### Example

Note that one must define $f$ and $p$ independently. That is, calling something like pullback(x^3+y^2, p) does not work (not sure why).

Set the function whose gradient is our target

In [66]:
f(x,y)= x^3+y^4;

Fix the transformation (monomial or not)

In [76]:
p(u,v)=(u^2*v,u^3*v^2);

Call the function, store the resulting vector in $V$ and print it

In [77]:
V=pullback(f,p);
print(V);

[-2*(2*conjugate(u)^5*conjugate(v)^4 - 3*u*v)/(u^2*v^2*conjugate(u)^8*conjugate(v)^6 + u^2*v^2*conjugate(u)^2*conjugate(v))]
[       (8*conjugate(u)^5*conjugate(v)^4 - 9*u*v)/(u^3*v*conjugate(u)^8*conjugate(v)^6 + u^3*v*conjugate(u)^2*conjugate(v))]


We can extract the denominator of the first entry of the vector and factorize it

In [78]:
factor(V[0,0].denominator())

(conjugate(u)^6*conjugate(v)^5 + 1)*u^2*v^2*conjugate(u)^2*conjugate(v)

The same for the second entry

In [79]:
factor(V[1,0].denominator())

(conjugate(u)^6*conjugate(v)^5 + 1)*u^3*v*conjugate(u)^2*conjugate(v)

And extract the common factor

In [82]:
common_denominator = gcd(factor(V[0,0].denominator()),factor(V[1,0].denominator()));
print(common_denominator);

(conjugate(u)^6*conjugate(v)^5 + 1)*u^2*v*conjugate(u)^2*conjugate(v)


Now we have a nice expression of the vector field as $1$ over the denominator above times the following nice column vector

In [97]:
print((common_denominator*V).simplify_full())

[-2*(2*conjugate(u)^5*conjugate(v)^4 - 3*u*v)/v]
[   (8*conjugate(u)^5*conjugate(v)^4 - 9*u*v)/u]


Notice the only denominators inside the vector are only $v$ and $u$ !

## 2. More complicated function using the Hessian

In [72]:
def pullback_c(f, p):
    g=f(p[0],p[1]);
    J=jacobian(p,(u,v));
    H=((J.H)*J);
    V=matrix(SR, 2, [conjugate(diff(g,u)),conjugate(diff(g,v))])
    G = (conjugate(g)^(-1))*(H^(-1)*V)
    return (G(u,v)).simplify_full()

Recall to define f and p before calling

In [73]:
f(x,y)= x^3+y^4;
p(u,v)=(u^2*v,u^3*v^2);

You can see it takes considerably more time to run this other function

In [74]:
pullback_c(f,p)

[-2*(2*conjugate(u)^5*conjugate(v)^4 - 3*u*v)/(u^2*v^2*conjugate(u)^8*conjugate(v)^6 + u^2*v^2*conjugate(u)^2*conjugate(v))]
[       (8*conjugate(u)^5*conjugate(v)^4 - 9*u*v)/(u^3*v*conjugate(u)^8*conjugate(v)^6 + u^3*v*conjugate(u)^2*conjugate(v))]

Now we can check that both functions coincide :)

In [75]:
pullback(f,p)==pullback_c(f,p)

True

## 3. Step-by-step (in case one wants to keep track of errors in handmade computations)

Here we just have a step-by-step computation of all the function appearing above in case one wants to keep track of errors

Fix f and p

In [160]:
f(x,y)= x^3+y^4;
p(u,v)=(u^2*v,u^3*v^2);

Set the function g as the pullback of f by p (and print its factorization)

In [161]:
g=f(p[0],p[1]);
print(g.factor());

(u^6*v^5 + 1)*u^6*v^3


Compute the jacobian of the monomial transformation (and print it)

In [162]:
J=jacobian(p,(u,v))
print(J)

[    (u, v) |--> 2*u*v       (u, v) |--> u^2]
[(u, v) |--> 3*u^2*v^2   (u, v) |--> 2*u^3*v]


Compute the complex gradient of f in the original space as a $2 \times 1$ symbolic matrix (and print it)

In [163]:
W = (conjugate(f)^(-1))*(matrix(SR, 2, [conjugate(diff(f,x)),conjugate(diff(f,y))]))
print(W(x,y).simplify_full())

[3*conjugate(x)^2/(conjugate(y)^4 + conjugate(x)^3)]
[4*conjugate(y)^3/(conjugate(y)^4 + conjugate(x)^3)]


Compute the pullback of the vector field by precomposing by $p$ and multiplying by the inverse of the jacobian

In [164]:
L=J(u,v)^(-1)*W(p[0],p[1]);

In [165]:
H=((J.H)*J);

In [166]:
print(H)

[(u, v) |--> 9*u^2*v^2*conjugate(u)^2*conjugate(v)^2 + 4*u*v*conjugate(u)*conjugate(v)   (u, v) |--> 6*u^3*v*conjugate(u)^2*conjugate(v)^2 + 2*u^2*conjugate(u)*conjugate(v)]
[             (u, v) |--> 6*u^2*v^2*conjugate(u)^3*conjugate(v) + 2*u*v*conjugate(u)^2                  (u, v) |--> 4*u^3*v*conjugate(u)^3*conjugate(v) + u^2*conjugate(u)^2]
